In [97]:
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
suppressPackageStartupMessages(source("tree_helper_function.R"))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(dplyr))
load("environment/brain_sim_nodtu/mode=gc_bias/mapDf.RData")
# load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/seBrainSim.RData")
trueCountsDir <- "/fs/cbcb-lab/rob/students/noor/Uncertainity/brain_sim_nodtu/mode=gc_bias/reads_fastq"

trueCountsDf <- lapply(seq(12), function(i) {
    samp <- ifelse(i<=6,i,i-6)
    rep <- ifelse(i<=6,1,2)
    file <- file.path(trueCountsDir, samp, paste(paste("true_counts", rep,sep="_"),"tsv",sep="."))
    df <- read.delim(file, row.names = 1)
})

comp_mard <- function(x, y, cutoff = 0.01) {
    if(length(x) != length(y))
        stop(paste(length(x), length(y)))
    relDiff <- rep(0, length(x))
    zInds <- which(x <= cutoff & y <= cutoff)
    print(length(zInds)/length(x))
    nzInds <- setdiff(seq_along(x), zInds)
    relDiff[nzInds] <- abs(x[nzInds] - y[nzInds])/(x[nzInds] + y[nzInds])
    return(sum(relDiff)/length(x))
}

In [98]:
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/seBrainSim.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/treeCons.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/treeCor.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/treeMC.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/treeMC0.RData")

In [99]:
dfCor <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfCor) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

dfPCor <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfPCor) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

dfMard <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfMard) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

#### Consensus

In [100]:
consTxps <- treeCons$tip.label
trueTxps <- Reduce(union, lapply(trueCountsDf, function(df) rownames(df)))
allTxps <- union(consTxps, trueTxps)
mb <- mergeLeaves(treeCons, seBrainSim[allTxps,])
yCons <- mb[["ySwish"]]
tr <- mb[["tree"]]
yAggCons <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=assays(yCons)[["counts"]])

[1] "Missing txps 925"


In [101]:
trueCountsDfU <- lapply(trueCountsDf, function(df) {
    mtnames <- setdiff(allTxps, rownames(df))
    rbind(df, data.frame(Counts=rep(0,length(mtnames)), row.names = mtnames))
})
rnames <- rownames(trueCountsDfU[[1]])
trueCountsDfU <- lapply(trueCountsDfU, function(df) {
    df <- df[rnames,]
    df
})
trueCountsDfU <- do.call(cbind,trueCountsDfU)

colnames(trueCountsDfU) <- colnames(seBrainSim)
rownames(trueCountsDfU) <- rnames

trueCountsDfU <- trueCountsDfU[tr$tip,]

trueCountsAgg <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=trueCountsDfU)

In [102]:
depth <- node.depth(tr,2)
depth[depth > 6] = 7
dfCor$Sample <- colnames(seBrainSim)
dfCor$Type <- rep("Consensus", ncol(seBrainSim))
dfMard$Sample <- colnames(seBrainSim)
dfMard$Type <- rep("Consensus", ncol(seBrainSim))

# head(dfCor)
mat <- matrix(0, ncol=ncol(seBrainSim), nrow=7)
for(i in seq(7)) {
    mat[i,] = diag(cor(trueCountsAgg[depth==i,], yAggCons[depth==i,], method = "spearman"))
    n <- paste("Depth", i, sep="_")
    dfCor[[n]] <- mat[i,]
    mat[i,] = diag(cor(trueCountsAgg[depth==i,], yAggCons[depth==i,]))
    dfMard[[n]] <- mat[i,]
}



In [103]:
depth <- node.depth(tr,2)
depth[depth > 6] = 7
diag(cor(trueCountsAgg[depth==1,], yAggCons[depth==1,]))

1_1       2_1       3_1       4_1       5_1       6_1       1_2       2_2 
0.9999769 0.9999661 0.9999662 0.9999370 0.9999748 0.9999807 0.9999474 0.9998258 
      3_2       4_2       5_2       6_2 
0.9999177 0.9999551 0.9999051 0.9998073

#### Correlation Tree

In [104]:
corTxps <- treeCor$tip.label
trueTxps <- Reduce(union, lapply(trueCountsDf, function(df) rownames(df)))
allTxps <- union(corTxps, trueTxps)
mb <- mergeLeaves(treeCor, seBrainSim[allTxps,])
yCor <- mb[["ySwish"]]
tr <- mb[["tree"]]
yAggCor <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=assays(yCor)[["counts"]])

[1] "Missing txps 925"


In [105]:
trueCountsDfU <- lapply(trueCountsDf, function(df) {
    mtnames <- setdiff(allTxps, rownames(df))
    rbind(df, data.frame(Counts=rep(0,length(mtnames)), row.names = mtnames))
})
rnames <- rownames(trueCountsDfU[[1]])
trueCountsDfU <- lapply(trueCountsDfU, function(df) {
    df <- df[rnames,]
    df
})
trueCountsDfU <- do.call(cbind,trueCountsDfU)

colnames(trueCountsDfU) <- colnames(seBrainSim)
rownames(trueCountsDfU) <- rnames

trueCountsDfU <- trueCountsDfU[tr$tip,]

trueCountsAgg <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=trueCountsDfU)

In [106]:
depth <- node.depth(tr,2)
depth[depth > 6] = 7

dfT <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfT) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

dfT2 <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfT2) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

dfT3 <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfT3) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")


dfT$Sample <- colnames(seBrainSim)
dfT$Type <- rep("Anti-Correlation", ncol(seBrainSim))
dfT2$Sample <- colnames(seBrainSim)
dfT2$Type <- rep("Anti-Correlation", ncol(seBrainSim))
# head(dfCor)
mat <- matrix(0, ncol=ncol(seBrainSim), nrow=7)
for(i in seq(7)) {
    mat[i,] = diag(cor(trueCountsAgg[depth==i,], yAggCor[depth==i,], method = "spearman"))
    n <- paste("Depth", i, sep="_")
    dfT[[n]] <- mat[i,]
    mat[i,] = sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[depth==i,j], yAggCor[depth==i,j]))
    dfT2[[n]] <- mat[i,]
}
colMeans(dfT[,2:8])
colMeans(dfT2[,2:8])

dfCor <- rbind(dfCor, dfT)
dfMard <- rbind(dfMard, dfT2)


[1] 0.486135
[1] 0.485918
[1] 0.4852521
[1] 0.4856636
[1] 0.4846759
[1] 0.4849902
[1] 0.486337
[1] 0.4858881
[1] 0.4848705
[1] 0.4853194
[1] 0.4850725
[1] 0.4849678
[1] 0.003990878
[1] 0.003720819
[1] 0.003750825
[1] 0.003930865
[1] 0.003540779
[1] 0.003990878
[1] 0.004080898
[1] 0.004020885
[1] 0.004050891
[1] 0.004140911
[1] 0.003750825
[1] 0.003960871
[1] 5.846586e-05
[1] 0
[1] 0.0001169317
[1] 0
[1] 5.846586e-05
[1] 5.846586e-05
[1] 0.0001169317
[1] 0.0002338634
[1] 0.0001169317
[1] 5.846586e-05
[1] 0.0001169317
[1] 0.0001753976
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0


Depth_1   Depth_2   Depth_3   Depth_4   Depth_5   Depth_6   Depth_7 
0.9448149 0.9913943 0.9977744 0.9994178 0.9997756 0.9998456 0.9999731

Depth_1     Depth_2     Depth_3     Depth_4     Depth_5     Depth_6 
0.099400957 0.068441162 0.029299666 0.013500659 0.006791058 0.004010136 
    Depth_7 
0.002000134

#### Mean Consensus

In [107]:
miTxps <- treeMC$tip.label
trueTxps <- Reduce(union, lapply(trueCountsDf, function(df) rownames(df)))
allTxps <- union(miTxps, trueTxps)
mb <- mergeLeaves(treeMC, seBrainSim[allTxps,])
yMI <- mb[["ySwish"]]
tr <- mb[["tree"]]
yAggMI <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=assays(yMI)[["counts"]])

[1] "Missing txps 925"


In [108]:
trueCountsDfU <- lapply(trueCountsDf, function(df) {
    mtnames <- setdiff(allTxps, rownames(df))
    rbind(df, data.frame(Counts=rep(0,length(mtnames)), row.names = mtnames))
})
rnames <- rownames(trueCountsDfU[[1]])
trueCountsDfU <- lapply(trueCountsDfU, function(df) {
    df <- df[rnames,]
    df
})
trueCountsDfU <- do.call(cbind,trueCountsDfU)

colnames(trueCountsDfU) <- colnames(seBrainSim)
rownames(trueCountsDfU) <- rnames

trueCountsDfU <- trueCountsDfU[tr$tip,]

trueCountsAgg <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=trueCountsDfU)

In [109]:
depth <- node.depth(tr,2)
depth[depth > 6] = 7

dfT <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfT) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

dfT2 <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfT2) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

dfT$Sample <- colnames(seBrainSim)
dfT$Type <- rep("Mean Inf", ncol(seBrainSim))
dfT2$Sample <- colnames(seBrainSim)
dfT2$Type <- rep("Mean Inf", ncol(seBrainSim))

# head(dfCor)
mat <- matrix(0, ncol=ncol(seBrainSim), nrow=7)
for(i in seq(7)) {
    mat[i,] = diag(cor(trueCountsAgg[depth==i,], yAggMI[depth==i,], method = "spearman"))
    n <- paste("Depth", i, sep="_")
    dfT[[n]] <- mat[i,]
    mat[i,] = sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[depth==i,j], yAggMI[depth==i,j]))
    dfT2[[n]] <- mat[i,]
    mat[i,] = diag(cor(trueCountsAgg[depth==i,], yAggMI[depth==i,]))
    dfT3[[n]] <- mat[i,]
}
colMeans(dfT[,2:8])
colMeans(dfT2[,2:8])

dfCor <- rbind(dfCor, dfT)
dfMard <- rbind(dfMard, dfT2)


[1] 0.486135
[1] 0.485918
[1] 0.4852521
[1] 0.4856636
[1] 0.4846759
[1] 0.4849902
[1] 0.486337
[1] 0.4858881
[1] 0.4848705
[1] 0.4853194
[1] 0.4850725
[1] 0.4849678
[1] 0.203475
[1] 0.2032449
[1] 0.2033983
[1] 0.2028613
[1] 0.2017106
[1] 0.2017874
[1] 0.2047024
[1] 0.2032065
[1] 0.2022093
[1] 0.2040887
[1] 0.2019024
[1] 0.2020942
[1] 0.1471943
[1] 0.1464265
[1] 0.1466627
[1] 0.1469581
[1] 0.1453633
[1] 0.1456586
[1] 0.1467218
[1] 0.1466037
[1] 0.1457177
[1] 0.1468399
[1] 0.1455995
[1] 0.1457767
[1] 0.1109586
[1] 0.1108778
[1] 0.1108778
[1] 0.1103933
[1] 0.110474
[1] 0.1103125
[1] 0.1116046
[1] 0.1107163
[1] 0.1109586
[1] 0.1108778
[1] 0.1102318
[1] 0.110151
[1] 0.08872618
[1] 0.08915689
[1] 0.08926456
[1] 0.08926456
[1] 0.08829547
[1] 0.08851082
[1] 0.08991063
[1] 0.08872618
[1] 0.08915689
[1] 0.08872618
[1] 0.08818779
[1] 0.0876494
[1] 0.0681915
[1] 0.0686177
[1] 0.06847564
[1] 0.06918596
[1] 0.0686177
[1] 0.06748118
[1] 0.06975423
[1] 0.06847564
[1] 0.0690439
[1] 0.06804944
[1] 0.068

Depth_1   Depth_2   Depth_3   Depth_4   Depth_5   Depth_6   Depth_7 
0.9448149 0.9976634 0.9991590 0.9994891 0.9996490 0.9997309 0.9999152

Depth_1    Depth_2    Depth_3    Depth_4    Depth_5    Depth_6    Depth_7 
0.09940096 0.03485119 0.02631151 0.02277984 0.01963624 0.01796507 0.01008245

#### MeanInf0

In [110]:
mi0Txps <- treeMC0$tip.label
trueTxps <- Reduce(union, lapply(trueCountsDf, function(df) rownames(df)))
allTxps <- union(mi0Txps, trueTxps)
mb <- mergeLeaves(treeMC0, seBrainSim[allTxps,])
yMI0 <- mb[["ySwish"]]
tr <- mb[["tree"]]
yAggMI0 <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=assays(yMI0)[["counts"]])

[1] "Missing txps 603"


In [111]:
trueCountsDfU <- lapply(trueCountsDf, function(df) {
    mtnames <- setdiff(allTxps, rownames(df))
    rbind(df, data.frame(Counts=rep(0,length(mtnames)), row.names = mtnames))
})
rnames <- rownames(trueCountsDfU[[1]])
trueCountsDfU <- lapply(trueCountsDfU, function(df) {
    df <- df[rnames,]
    df
})
trueCountsDfU <- do.call(cbind,trueCountsDfU)

colnames(trueCountsDfU) <- colnames(seBrainSim)
rownames(trueCountsDfU) <- rnames

trueCountsDfU <- trueCountsDfU[tr$tip,]

trueCountsAgg <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=trueCountsDfU)

In [112]:
depth <- node.depth(tr,2)
depth[depth > 6] = 7

dfT <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfT) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

dfT2 <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfT2) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

dfT$Sample <- colnames(seBrainSim)
dfT$Type <- rep("Mean Inf0", ncol(seBrainSim))
dfT2$Sample <- colnames(seBrainSim)
dfT2$Type <- rep("Mean Inf0", ncol(seBrainSim))
# head(dfCor)
mat <- matrix(0, ncol=ncol(seBrainSim), nrow=7)
for(i in seq(7)) {
    mat[i,] = diag(cor(trueCountsAgg[depth==i,], yAggMI0[depth==i,], method = "spearman"))
    n <- paste("Depth", i, sep="_")
    dfT[[n]] <- mat[i,]
    mat[i,] = sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[depth==i,j], yAggMI0[depth==i,j]))
    dfT2[[n]] <- mat[i,]
}
colMeans(dfT[,2:8])
colMeans(dfT2[,2:8])

dfCor <- rbind(dfCor, dfT)
dfMard <- rbind(dfMard, dfT2)


[1] 0.508769
[1] 0.5085474
[1] 0.5078968
[1] 0.5083186
[1] 0.5073034
[1] 0.5076465
[1] 0.5089049
[1] 0.5085188
[1] 0.5076108
[1] 0.5080183
[1] 0.507811
[1] 0.5076465
[1] 0.02097721
[1] 0.02097721
[1] 0.02056182
[1] 0.02082143
[1] 0.02004258
[1] 0.02014643
[1] 0.02149644
[1] 0.02087336
[1] 0.0203022
[1] 0.02128875
[1] 0.01983488
[1] 0.02004258
[1] 0.005114736
[1] 0.004700028
[1] 0.004769146
[1] 0.005045618
[1] 0.004700028
[1] 0.004561792
[1] 0.005252972
[1] 0.004907382
[1] 0.004838264
[1] 0.005114736
[1] 0.004492673
[1] 0.004354437
[1] 0.001879085
[1] 0.001470588
[1] 0.001388889
[1] 0.002124183
[1] 0.001797386
[1] 0.001633987
[1] 0.002205882
[1] 0.001715686
[1] 0.001797386
[1] 0.001715686
[1] 0.001388889
[1] 0.001470588
[1] 0.001417769
[1] 0.0008506616
[1] 0.0009451796
[1] 0.001323251
[1] 0.001228733
[1] 0.001323251
[1] 0.001701323
[1] 0.001134216
[1] 0.001323251
[1] 0.001134216
[1] 0.001039698
[1] 0.0008506616
[1] 0.0009933775
[1] 0.0005518764
[1] 0.0005518764
[1] 0.0009933775
[1] 0.00

Depth_1   Depth_2   Depth_3   Depth_4   Depth_5   Depth_6   Depth_7 
0.9446949 0.9986109 0.9996749 0.9998361 0.9998941 0.9999174 0.9999696

Depth_1     Depth_2     Depth_3     Depth_4     Depth_5     Depth_6 
0.095208593 0.025189640 0.011037425 0.006958612 0.004819988 0.003715719 
    Depth_7 
0.001796846

In [114]:
dfCor %>% group_by(Type) %>% select(starts_with("Depth")) %>% summarise_all(.funs = c(mean="mean"))
dfMard %>% group_by(Type) %>% select(starts_with("Depth")) %>% summarise_all(.funs = c(mean="mean"))


Adding missing grouping variables: `Type`


Type,Depth_1_mean,Depth_2_mean,Depth_3_mean,Depth_4_mean,Depth_5_mean,Depth_6_mean,Depth_7_mean
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Anti-Correlation,0.9448149,0.9913943,0.9977744,0.9994178,0.9997756,0.9998456,0.9999731
Consensus,0.9448149,0.9910081,0.9955883,0.9976570,0.9986886,0.9992657,0.9997477
Mean Inf,0.9448149,0.9976634,0.9991590,0.9994891,0.9996490,0.9997309,0.9999152
Mean Inf0,0.9446949,0.9986109,0.9996749,0.9998361,0.9998941,0.9999174,0.9999696


Adding missing grouping variables: `Type`


Type,Depth_1_mean,Depth_2_mean,Depth_3_mean,Depth_4_mean,Depth_5_mean,Depth_6_mean,Depth_7_mean
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Anti-Correlation,0.09940096,0.06844116,0.02929967,0.013500659,0.006791058,0.004010136,0.002000134
Consensus,0.99993000,0.99955032,0.99992230,0.999961021,0.999964303,0.999986354,0.999999998
Mean Inf,0.09940096,0.03485119,0.02631151,0.022779845,0.019636236,0.017965069,0.010082454
Mean Inf0,0.09520859,0.02518964,0.01103742,0.006958612,0.004819988,0.003715719,0.001796846
